# Data-prep

To start the second weekly project in the Data bootcamp, I have chosen a dataset with information on the happiness across 156 countries.

With this dataset, I want to test the hypothesis that happiness is much less correlated to monetary data like GDP per capita or social support and more dependant on non-monetary factors like generosity, healthy life expectancy, or freedom. Additionally, I will look for new data in the API and try to find additional correlations with the happiness index.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-data-and-functions" data-toc-modified-id="Import-libraries,-data-and-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, data and functions</a></span></li><li><span><a href="#Explore-dataset" data-toc-modified-id="Explore-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Explore dataset</a></span></li><li><span><a href="#Treat-dataset" data-toc-modified-id="Treat-dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Treat dataset</a></span><ul class="toc-item"><li><span><a href="#Clean-dataset" data-toc-modified-id="Clean-dataset-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Clean dataset</a></span></li><li><span><a href="#Import-data-from-API" data-toc-modified-id="Import-data-from-API-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Import data from API</a></span></li><li><span><a href="#Combine-datasets-and-clean" data-toc-modified-id="Combine-datasets-and-clean-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Combine datasets and clean</a></span></li></ul></li></ul></div>

## Import libraries, data and functions

Import libraries

In [1]:
import pandas as pd
import os
import requests
import json
import wbdata
import datetime

Import data

In [2]:
data = pd.read_csv("../data/2019.csv")

Import functions

In [3]:
import sys

sys.path.append('../src')

from happiness_report_functions import *

## Explore dataset

In [4]:
data

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
...,...,...,...,...,...,...,...,...,...
151,152,Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411
152,153,Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147
153,154,Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025
154,155,Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035


## Treat dataset

### Clean dataset

We will use defined functions to clean the dataset automatically. The required and optional inputs are:
- dataframe
- old_col_name and new_col_name in case any names want to be changed
- new_index_col_name in case we want to change the index

In [5]:
old_col_name = ['score']
new_col_name = ['happiness_score']
new_index_col_name = 'country_or_region'
col_to_drop = ['overall_rank']

In [6]:
clean_data_set(data, old_col_name = old_col_name, new_col_name = new_col_name, new_index_col_name = new_index_col_name, col_to_drop = col_to_drop)

In [7]:
data.head()

,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
country_or_region,,,,,,,
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


We will revisit the cleaning step once we have added the data from the API as there will probably be some need to clean the data

### Import data from API

I will use the World Bank API to import data that complements my current dataset

First I will look at what databases are available

In [8]:
print_WB_available_sources()

1 Doing Business
2 World Development Indicators
3 Worldwide Governance Indicators
5 Subnational Malnutrition Database
6 International Debt Statistics
11 Africa Development Indicators
12 Education Statistics
13 Enterprise Surveys
14 Gender Statistics
15 Global Economic Monitor
16 Health Nutrition and Population Statistics
18 IDA Results Measurement System
19 Millennium Development Goals
20 Quarterly Public Sector Debt
22 Quarterly External Debt Statistics SDDS
23 Quarterly External Debt Statistics GDDS
24 Poverty and Equity
25 Jobs
27 Global Economic Prospects
28 Global Financial Inclusion
29 The Atlas of Social Protection: Indicators of Resilience and Equity
30 Exporter Dynamics Database – Indicators at Country-Year Level
31 Country Policy and Institutional Assessment
32 Global Financial Development
33 G20 Financial Inclusion Indicators
34 Global Partnership for Education
35 Sustainable Energy for All
36 Statistical Capacity Indicators
37 LAC Equity Lab
38 Subnational Poverty
39 Health

I will chose 'World Development Indicators'. Now, using the API I can look at a list of all available indicators

In [9]:
show_WB_indicators('2')


Since I have commented the 'return' from the show_WB_indicators to avoid for a 'print' that is exrtremely long, I need to execute the code in the function outside of the function so I can reference the output later

In [10]:
source_id = '2'
indicators = requests.get(f"http://api.worldbank.org/v2/indicator?format=json&source={source_id}&per_page=1500")
indicatorsJSON = indicators.json()
wdev_indicators = dict()

# Add the Indicator IDs and Names to a dictionary
for i in indicatorsJSON[1]:
    key = i['id']
    value = i['name']
    wdev_indicators[key] = value

I will now create a list with all available countries so that I can get a feel of how 'rich' my new data will be

In [11]:
print_available_countries_WB()

[('ABW', 'Aruba'), ('AFG', 'Afghanistan'), ('AFR', 'Africa'), ('AGO', 'Angola'), ('ALB', 'Albania'), ('AND', 'Andorra'), ('ARB', 'Arab World'), ('ARE', 'United Arab Emirates'), ('ARG', 'Argentina'), ('ARM', 'Armenia'), ('ASM', 'American Samoa'), ('ATG', 'Antigua and Barbuda'), ('AUS', 'Australia'), ('AUT', 'Austria'), ('AZE', 'Azerbaijan'), ('BDI', 'Burundi'), ('BEA', 'East Asia & Pacific (IBRD-only countries)'), ('BEC', 'Europe & Central Asia (IBRD-only countries)'), ('BEL', 'Belgium'), ('BEN', 'Benin')]


Since I have commented the 'return' from the sprint_available_countries to avoid for a 'print' that is exrtremely long, I need to execute the code in the function outside of the function so I can reference the output later

In [12]:
country_ids = requests.get('http://api.worldbank.org/v2/country?format=json&per_page=250')
country_idsJSON = country_ids.json()
list_country_ids = []
list_country_ids_name = []

for i in country_idsJSON[1]:
    c_id = (i['id'])
    c_id_n = (i['id'], i['name'])
    list_country_ids.append(c_id)
    list_country_ids_name.append(c_id_n)

Now that I am ready, the World Bank library allows me to download a Pandas dataframe with the indicators I want, for the countries I need and the dates I like. 

In [13]:
indic = {
    'EG.ELC.ACCS.ZS': wdev_indicators.get('EG.ELC.ACCS.ZS'),
    'MS.MIL.TOTL.TF.ZS': wdev_indicators.get('MS.MIL.TOTL.TF.ZS'),
    'SL.EMP.SMGT.FE.ZS': wdev_indicators.get('SL.EMP.SMGT.FE.ZS'),
    'IT.NET.BBND.P2': wdev_indicators.get('IT.NET.BBND.P2'),
    'AG.LND.FRST.ZS': wdev_indicators.get('AG.LND.FRST.ZS'),
    'SM.POP.TOTL.ZS': wdev_indicators.get('SM.POP.TOTL.ZS'),
    'EN.URB.MCTY.TL.ZS': wdev_indicators.get('EN.URB.MCTY.TL.ZS'),
    'SH.PRV.SMOK': wdev_indicators.get('SH.PRV.SMOK'),
    'SH.ALC.PCAP.LI': wdev_indicators.get('SH.ALC.PCAP.LI')
        }
location_selection = list_country_ids
time_selection = datetime.datetime(2018, 1, 1), datetime.datetime(2018, 1, 1)
world_dev_ind = '2'

In [14]:
# Making the API call and assigning the resulting DataFrame to "datafr"
datafr = wbdata.get_dataframe(indicators = indic,
                            source = world_dev_ind,
                            country = location_selection, 
                            data_date = time_selection)

In [15]:
datafr.head()

,Access to electricity (% of population),Armed forces personnel (% of total labor force),Female share of employment in senior and middle management (%),Fixed broadband subscriptions (per 100 people),Forest area (% of land area),International migrant stock (% of population),Population in urban agglomerations of more than 1 million (% of total population),Prevalence of current tobacco use (% of adults),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)"
country,,,,,,,,,
Aruba,100.000000,NaN,NaN,18.209500,None,None,NaN,NaN,NaN
Afghanistan,98.713203,2.640774,NaN,0.047054,None,None,10.792339,NaN,0.21
Angola,43.259258,0.920862,NaN,0.355605,None,None,25.232912,NaN,6.94
Albania,100.000000,0.682841,31.6,12.514500,None,None,NaN,29.2,7.17
Andorra,100.000000,NaN,NaN,46.312000,None,None,NaN,33.8,11.02


We now have a dataframe downloaded from the World Bank API that we can merge with the dataset downloaded from Kaggle

### Combine datasets and clean

We will combine both dataframes using the index column to match results. As concat does an outer join by default, we won't lose any rows in this step

In [16]:
frames = [data, datafr]
combined_df = pd.concat(frames, axis = 1) 

In [17]:
combined_df

,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,Access to electricity (% of population),Armed forces personnel (% of total labor force),Female share of employment in senior and middle management (%),Fixed broadband subscriptions (per 100 people),Forest area (% of land area),International migrant stock (% of population),Population in urban agglomerations of more than 1 million (% of total population),Prevalence of current tobacco use (% of adults),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)"
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,100.000000,0.918790,32.009998,31.452700,None,None,23.190830,19.700000,10.780000
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,100.000000,0.480968,26.330000,44.059300,None,None,22.797877,18.600000,10.260000
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,100.000000,0.830370,33.470001,41.336300,None,None,19.055742,13.000000,7.410000
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,100.000000,NaN,43.049999,40.555600,None,None,NaN,13.800000,9.120000
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,100.000000,0.448746,24.760000,43.084300,None,None,12.415951,23.400000,9.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.660057,0.397188,NaN,0.415993,None,None,15.623313,11.448154,5.837431
Small states,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.658756,1.047015,NaN,8.471551,None,None,NaN,21.321521,5.104701
Sao Tome and Principe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.000000,NaN,NaN,0.737817,None,None,NaN,5.400000,5.880000
Suriname,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.400000,0.856192,NaN,12.704400,None,None,NaN,NaN,5.330000


Now we will clean this combines dataframe. We don't want to change any column names of re-index

In [18]:
clean_data_set(combined_df)

In [19]:
combined_df

,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,access_to_electricity_(%_of_population),armed_forces_personnel_(%_of_total_labor_force),female_share_of_employment_in_senior_and_middle_management_(%),fixed_broadband_subscriptions_(per_100_people),population_in_urban_agglomerations_of_more_than_1_million_(%_of_total_population),prevalence_of_current_tobacco_use_(%_of_adults),"total_alcohol_consumption_per_capita_(liters_of_pure_alcohol,_projected_estimates,_15+_years_of_age)"
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,100.000000,0.918790,32.009998,31.452700,23.190830,19.700000,10.780000
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,100.000000,0.480968,26.330000,44.059300,22.797877,18.600000,10.260000
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,100.000000,0.830370,33.470001,41.336300,19.055742,13.000000,7.410000
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,100.000000,NaN,43.049999,40.555600,NaN,13.800000,9.120000
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,100.000000,0.448746,24.760000,43.084300,12.415951,23.400000,9.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.660057,0.397188,NaN,0.415993,15.623313,11.448154,5.837431
Small states,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.658756,1.047015,NaN,8.471551,NaN,21.321521,5.104701
Sao Tome and Principe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.000000,NaN,NaN,0.737817,NaN,5.400000,5.880000
Suriname,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.400000,0.856192,NaN,12.704400,NaN,NaN,5.330000


Next, since we are focusing on the happiness score, those regions that do not have a happiness score will not be used and can therefore be deleted

In [20]:
combined_df = combined_df[combined_df['happiness_score'].notna()]

In [21]:
combined_df

,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,access_to_electricity_(%_of_population),armed_forces_personnel_(%_of_total_labor_force),female_share_of_employment_in_senior_and_middle_management_(%),fixed_broadband_subscriptions_(per_100_people),population_in_urban_agglomerations_of_more_than_1_million_(%_of_total_population),prevalence_of_current_tobacco_use_(%_of_adults),"total_alcohol_consumption_per_capita_(liters_of_pure_alcohol,_projected_estimates,_15+_years_of_age)"
Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,100.000000,0.918790,32.009998,31.452700,23.190830,19.7,10.78
Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,100.000000,0.480968,26.330000,44.059300,22.797877,18.6,10.26
Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,100.000000,0.830370,33.470001,41.336300,19.055742,13.0,7.41
Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,100.000000,NaN,43.049999,40.555600,NaN,13.8,9.12
Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,100.000000,0.448746,24.760000,43.084300,12.415951,23.4,9.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411,34.716572,0.566503,33.220001,0.060974,8.598937,13.3,8.95
Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025,98.713203,2.640774,NaN,0.047054,10.792339,NaN,0.21
Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035,32.420399,0.542461,NaN,0.013029,NaN,NaN,2.38


The final step is to save this information in a csv file so it can be used later

In [22]:
save_df_as_csv(combined_df, file_name = 'happiness_combined_df')

0

Now we are ready for the visualization process to test our hypothesis